In [1]:
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
import urllib, json
import pandas as pd
import mxnet as mx
from mxnet import nd, init, autograd
from mxnet.gluon import nn
from mxnet import gluon
from time import time
import re

/home/chao2/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#load training data
train_input_temp = pd.read_csv('EI_training_input_with_header.csv',header=None)
train_input_temp2 = pd.read_csv('EI_training_input.csv',header=None)

#train_output_temp = pd.read_csv('EI_training_output_data.csv')
train_output_temp2 = pd.read_csv('EI_training_output.csv',header=None)

In [3]:
#train_input_temp

In [4]:
train_input=nd.array(train_input_temp2,mx.gpu())
#train_input

In [5]:
train_output=nd.array(train_output_temp2,mx.gpu())#
train_output[[1,5],:]


[[0. 0. 1.]
 [0. 0. 1.]]
<NDArray 2x3 @gpu(0)>

In [6]:
ctx = mx.gpu()
layer = nn.Sequential()

with layer.name_scope():
    layer.add(        
        nn.Dense(6, activation="relu"),
        nn.Dropout(0.5),          # dropout to ease the overfitting problem
        nn.Dense(3)
    )
layer.initialize(ctx=ctx)
x = nd.random.uniform(0,1,(2,train_input.shape[1]),ctx=ctx)
layer(x)


[[ 0.00341021  0.00367314 -0.00375737]
 [ 0.00521525  0.00640165 -0.00486244]]
<NDArray 2x3 @gpu(0)>

In [7]:
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss(sparse_label=False)
l2loss = gluon.loss.L2Loss()
kldivloss = gluon.loss.KLDivLoss(from_logits=False)
trainer = gluon.Trainer(layer.collect_params(),
                        'adam', {'learning_rate': 0.1,'wd': 0.0001}) # 'weight decay to ease the overfitting problem
#trainer = gluon.Trainer(layer.collect_params(), 'sgd',
                            #{'learning_rate': 0.1, 'momentum': 0.95})
train_loss, train_acc, valid_acc = 0., 0., 0.

In [8]:
embed2=[[0,0,1],[0,1,0],[1,0,0]]
def acc(output, label):
    # output: (batch, num_output) float32 ndarray
    # label: (batch, ) int32 ndarray
    return (output.argmax(axis=1) ==
            label.argmax(axis=1)).mean().asscalar()
def acc2(output, label):
    # output: (batch, num_output) float32 ndarray
    # label: (batch, ) int32 ndarray
    return (output.argmax(axis=1) -
            label.argmax(axis=1)).abs().mean().asscalar()

In [9]:
train_number = 36
print('tain number', train_number,'over',train_input.shape[0])
batchsize=36
sampler = gluon.data.RandomSampler(train_number)
batch_sampler = gluon.data.BatchSampler(sampler, batchsize, 'keep')
print(list(batch_sampler))

tain number 36 over 44
[[23, 26, 22, 9, 6, 8, 15, 34, 29, 3, 33, 25, 2, 20, 19, 16, 24, 1, 18, 7, 0, 12, 27, 10, 5, 30, 32, 35, 21, 31, 4, 14, 28, 17, 11, 13]]


In [10]:
for epoch in range(60):
    train_loss, train_acc, valid_acc = 0., 0., 0.
    tic = time()
    cumulative_loss=0
    for b in list(batch_sampler):
        #print(b)
        with autograd.record():
            output = layer(train_input[b,:])
            loss = kldivloss(output, train_output[b,:])
            #loss = softmax_cross_entropy(output,train_output[b,:])
        loss.backward()
        train_loss+=loss.mean().asscalar()
        trainer.step(batchsize)
    # update parameters
    # calculate training metrics
    train_acc = acc(layer(train_input[0:train_number,:]), train_output[0:train_number,:])

    # calculate validation accuracy
    valid_acc = acc(layer(train_input[train_number:,:]), train_output[train_number:,:])

    print("# %d: Loss: %.5f, Acc. Train %.3f, Acc. Test %.3f, Time %.1f sec" % (
         epoch, train_loss,
         train_acc,
         valid_acc,
         time()-tic))

# 0: Loss: 0.36577, Acc. Train 0.472, Acc. Test 0.625, Time 0.0 sec
# 1: Loss: 0.34623, Acc. Train 0.472, Acc. Test 0.625, Time 0.0 sec
# 2: Loss: 0.36417, Acc. Train 0.472, Acc. Test 0.625, Time 0.0 sec
# 3: Loss: 0.33220, Acc. Train 0.472, Acc. Test 0.625, Time 0.0 sec
# 4: Loss: 0.31502, Acc. Train 0.667, Acc. Test 0.500, Time 0.0 sec
# 5: Loss: 0.31584, Acc. Train 0.694, Acc. Test 0.500, Time 0.0 sec
# 6: Loss: 0.32984, Acc. Train 0.694, Acc. Test 0.500, Time 0.0 sec
# 7: Loss: 0.28488, Acc. Train 0.667, Acc. Test 0.500, Time 0.0 sec
# 8: Loss: 0.27904, Acc. Train 0.667, Acc. Test 0.625, Time 0.0 sec
# 9: Loss: 0.29561, Acc. Train 0.694, Acc. Test 0.500, Time 0.0 sec
# 10: Loss: 0.28004, Acc. Train 0.806, Acc. Test 0.625, Time 0.0 sec
# 11: Loss: 0.27220, Acc. Train 0.833, Acc. Test 0.750, Time 0.0 sec
# 12: Loss: 0.28692, Acc. Train 0.806, Acc. Test 0.750, Time 0.0 sec
# 13: Loss: 0.28091, Acc. Train 0.694, Acc. Test 0.375, Time 0.0 sec
# 14: Loss: 0.25644, Acc. Train 0.722, Acc. 

In [11]:
# 0: Loss: 0.36589, Acc. Train 0.444, Acc. Test 0.714, Time 0.0 sec
# 1: Loss: 0.35063, Acc. Train 0.444, Acc. Test 0.714, Time 0.0 sec
# 2: Loss: 0.36259, Acc. Train 0.500, Acc. Test 0.714, Time 0.0 sec
# 3: Loss: 0.33976, Acc. Train 0.556, Acc. Test 0.857, Time 0.0 sec
# 4: Loss: 0.33622, Acc. Train 0.556, Acc. Test 0.429, Time 0.0 sec
# 5: Loss: 0.33215, Acc. Train 0.694, Acc. Test 0.429, Time 0.0 sec
# 6: Loss: 0.31903, Acc. Train 0.806, Acc. Test 0.714, Time 0.0 sec
# 7: Loss: 0.30061, Acc. Train 0.694, Acc. Test 1.000, Time 0.0 sec
# 8: Loss: 0.28325, Acc. Train 0.694, Acc. Test 0.714, Time 0.0 sec
# 9: Loss: 0.26440, Acc. Train 0.750, Acc. Test 0.714, Time 0.0 sec
# 10: Loss: 0.24668, Acc. Train 0.750, Acc. Test 0.714, Time 0.0 sec
# 11: Loss: 0.23730, Acc. Train 0.778, Acc. Test 0.714, Time 0.0 sec
# 12: Loss: 0.22289, Acc. Train 0.722, Acc. Test 0.857, Time 0.0 sec
# 13: Loss: 0.21040, Acc. Train 0.750, Acc. Test 0.857, Time 0.0 sec
# 14: Loss: 0.20170, Acc. Train 0.750, Acc. Test 0.714, Time 0.0 sec
# 15: Loss: 0.19095, Acc. Train 0.750, Acc. Test 0.714, Time 0.0 sec
# 16: Loss: 0.18502, Acc. Train 0.694, Acc. Test 0.714, Time 0.0 sec
# 17: Loss: 0.17856, Acc. Train 0.750, Acc. Test 0.714, Time 0.0 sec
# 18: Loss: 0.17327, Acc. Train 0.750, Acc. Test 0.714, Time 0.0 sec
# 19: Loss: 0.17310, Acc. Train 0.722, Acc. Test 0.714, Time 0.0 sec
# 20: Loss: 0.17011, Acc. Train 0.806, Acc. Test 0.571, Time 0.0 sec
# 21: Loss: 0.16822, Acc. Train 0.750, Acc. Test 0.714, Time 0.0 sec
# 22: Loss: 0.16849, Acc. Train 0.750, Acc. Test 0.714, Time 0.0 sec
# 23: Loss: 0.16630, Acc. Train 0.750, Acc. Test 0.714, Time 0.0 sec
# 24: Loss: 0.16510, Acc. Train 0.722, Acc. Test 0.714, Time 0.0 sec
# 25: Loss: 0.16239, Acc. Train 0.750, Acc. Test 0.714, Time 0.0 sec
# 26: Loss: 0.16102, Acc. Train 0.750, Acc. Test 0.714, Time 0.0 sec
# 27: Loss: 0.15868, Acc. Train 0.750, Acc. Test 0.714, Time 0.0 sec
# 28: Loss: 0.15671, Acc. Train 0.750, Acc. Test 0.714, Time 0.0 sec
# 29: Loss: 0.15614, Acc. Train 0.750, Acc. Test 0.714, Time 0.0 sec
# 30: Loss: 0.15362, Acc. Train 0.750, Acc. Test 0.714, Time 0.0 sec
# 31: Loss: 0.15171, Acc. Train 0.778, Acc. Test 0.714, Time 0.0 sec
# 32: Loss: 0.14932, Acc. Train 0.750, Acc. Test 0.714, Time 0.0 sec
# 33: Loss: 0.14809, Acc. Train 0.778, Acc. Test 0.714, Time 0.0 sec
# 34: Loss: 0.14746, Acc. Train 0.778, Acc. Test 0.714, Time 0.0 sec
# 35: Loss: 0.14706, Acc. Train 0.806, Acc. Test 0.571, Time 0.0 sec
# 36: Loss: 0.14664, Acc. Train 0.750, Acc. Test 0.714, Time 0.0 sec
# 37: Loss: 0.14592, Acc. Train 0.750, Acc. Test 0.714, Time 0.0 sec
# 38: Loss: 0.14584, Acc. Train 0.778, Acc. Test 0.714, Time 0.0 sec
# 39: Loss: 0.14441, Acc. Train 0.806, Acc. Test 0.714, Time 0.0 sec
# 40: Loss: 0.14368, Acc. Train 0.806, Acc. Test 0.714, Time 0.0 sec
# 41: Loss: 0.14253, Acc. Train 0.778, Acc. Test 0.714, Time 0.0 sec
# 42: Loss: 0.14097, Acc. Train 0.806, Acc. Test 0.714, Time 0.0 sec
# 43: Loss: 0.13925, Acc. Train 0.861, Acc. Test 0.571, Time 0.0 sec
# 44: Loss: 0.13772, Acc. Train 0.861, Acc. Test 0.571, Time 0.0 sec
# 45: Loss: 0.13668, Acc. Train 0.889, Acc. Test 0.571, Time 0.0 sec
# 46: Loss: 0.13575, Acc. Train 0.889, Acc. Test 0.571, Time 0.0 sec
# 47: Loss: 0.13470, Acc. Train 0.861, Acc. Test 0.571, Time 0.0 sec
# 48: Loss: 0.13335, Acc. Train 0.806, Acc. Test 0.714, Time 0.0 sec
# 49: Loss: 0.13194, Acc. Train 0.833, Acc. Test 0.714, Time 0.0 sec
# 50: Loss: 0.13045, Acc. Train 0.833, Acc. Test 0.714, Time 0.0 sec
# 51: Loss: 0.12892, Acc. Train 0.833, Acc. Test 0.714, Time 0.0 sec
# 52: Loss: 0.12766, Acc. Train 0.833, Acc. Test 0.714, Time 0.0 sec
# 53: Loss: 0.12623, Acc. Train 0.833, Acc. Test 0.714, Time 0.0 sec
# 54: Loss: 0.12491, Acc. Train 0.889, Acc. Test 0.571, Time 0.0 sec
# 55: Loss: 0.12360, Acc. Train 0.889, Acc. Test 0.571, Time 0.0 sec
# 56: Loss: 0.12216, Acc. Train 0.861, Acc. Test 0.571, Time 0.0 sec
# 57: Loss: 0.12042, Acc. Train 0.861, Acc. Test 0.571, Time 0.0 sec
# 58: Loss: 0.11782, Acc. Train 0.889, Acc. Test 0.571, Time 0.0 sec
# 59: Loss: 0.11521, Acc. Train 0.889, Acc. Test 0.571, Time 0.0 sec


In [12]:
layer.save_parameters('audit_deltaEI.params')
train_input_temp.to_pickle("audit_deltaEI.data")

In [13]:
# ['UZB', 'ZAF', 'ISR', 'MAR', 'KHM', 'SEN']: 'audit_deltaEI_095_1.params', "audit_deltaEI_095_1.data";# 49: Loss: 0.21741, Acc. Train 0.950, Acc. Test 1.000, Time 0.0 sec